In [2]:
import torch
import torch.nn.functional as f
import torchvision
import torchvision.datasets as datasets
from torchvision.transforms import transforms
from torch import nn,optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import random_split


In [13]:
class lightning(nn.Module):
  def __init__(self,input_size,num_classes):
    super().__init__()
    self.fc1=nn.Linear(input_size,50)
    self.fc2=nn.Linear(50, num_classes)

  def forward(self,x):
    x=f.relu(self.fc1(x))
    x=self.fc2(x)

    return x

In [4]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3


In [7]:
entire_dataset=datasets.MNIST(root="dir",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

train_ds,val_ds=random_split(entire_dataset, [50000, 10000])

test_ds = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2886625.65it/s]


Extracting dir/MNIST/raw/train-images-idx3-ubyte.gz to dir/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 506475.18it/s]


Extracting dir/MNIST/raw/train-labels-idx1-ubyte.gz to dir/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4475132.02it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3061801.47it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [8]:
train_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=batch_size, shuffle=False)

In [14]:
model_0=lightning(input_size=input_size,num_classes=num_classes)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_0.parameters(), lr=learning_rate)


In [16]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1)

        # Forward
        scores = model_0(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:

            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)

            # Get to correct shape
            x = x.reshape(x.shape[0], -1)

            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)

            # Check how many we got correct
            num_correct += (predictions == y).sum()

            # Keep track of number of samples
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

100%|██████████| 782/782 [00:08<00:00, 91.47it/s] 


In [20]:
model_0.to(device)
print(f"Accuracy on training set: {check_accuracy(train_loader, model_0)*100:.2f}")
print(f"Accuracy on validation set: {check_accuracy(val_loader, model_0)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_0)*100:.2f}")

Accuracy on training set: 10.98
Accuracy on validation set: 10.98
Accuracy on test set: 11.08
